# CS-GY 9223-D: Deep Learning Homework 1
Due on Friday, 15th February 2019, 11:55 PM

This homework can be done in pairs. Everyone must submit on NYU Classes individually.

Write down the UNIs (NetIDs) of your group (if applicable)

Member 1: Hupo Tang, ht1073

Member 2: Yuan Tang Lin, ytl329

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

## Data Loading

In [2]:
import torch
import torchvision
from torchvision.transforms import transforms

In [3]:
# Download the data

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


## Model Architecture

In [44]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        # here layer_dimensions is [3072, 256, 128, 10], 2 hidden layers
        # I set num_layers to 2, except input layer and output layer which are No.0,3 layer
        np.random.seed(1)
        
        self.batch_size = 100
        self.devide_ratio = 0.9
        self.alpha = 1e-1
        self.eps = 1e-20
        
        self.num_layers = len(layer_dimensions) - 2
        self.drop_prob = drop_prob
        self.reg_lambda = reg_lambda
        
        self.parameters = {} # save layer No.1,2,3 's parameters
        for i in range(1, len(layer_dimensions)):
            eps = np.sqrt(2.0 / (layer_dimensions[i] + layer_dimensions[i-1]))
            W = np.random.randn(layer_dimensions[i], layer_dimensions[i-1]) * eps
            b = np.zeros((layer_dimensions[i],))+1e-2
            self.parameters.update({i: {'W': W, 'b': b}})
                
        self.d_w,self.d_b=[0.9]*(self.num_layers+1),[0.9]*(self.num_layers+1)
        self.delta_w,self.delta_b=[0.0]*(self.num_layers+1),[0.0]*(self.num_layers+1)
        self.Egt_w,self.Egt_b=[0]*(self.num_layers+1),[0]*(self.num_layers+1)
        self.Edt_w,self.Edt_b=[0]*(self.num_layers+1),[0]*(self.num_layers+1)
        
        self.vw=[0]*(self.num_layers+1)
        self.vb=[0]*(self.num_layers+1)
        self.mw=[0]*(self.num_layers+1)
        self.mb=[0]*(self.num_layers+1)
        
        self.b1=0.9
        self.b2=0.995
        
        # init parameters
        

    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass. W_i, A_(i-1), b_i
        """
        Z = W.dot(A) + np.tile(b, (A.shape[1], 1)).T
        cache = {'Z': Z, 'A': A, 'W': W, 'b': b}
        if self.drop_prob > 0:
            Z, m = self.dropout(Z, self.drop_prob)
            cache['m'] = m

#         print(Z)
        return Z, cache
        

    def activationForward(self, A, activation="relu"):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        return self.relu(A)


    def relu(self, X):
        return np.maximum(X, 0)

            
    def dropout(self, A, prob):
        """
        :param A: Activation
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """
        M = np.random.rand(A.shape[0], A.shape[1])
        M = (M >= prob) * 1.0
        M /= (1 - prob)
        A = np.multiply(A, M)
        return A, M
    
    def softmax(self, X):
#         X = (X - X.mean())/(X.max()-X.min())
        return np.exp(X) / np.sum(np.exp(X), axis=0)
    
    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        AL = X
        cache = {}
        for l in range(1, self.num_layers+1):
            W = self.parameters[l]['W']
            b = self.parameters[l]['b']
            Z, _cache = self.affineForward(AL, W, b)
            AL = self.activationForward(Z, activation="relu")
            cache.update({l:_cache})
        last_W, last_b = self.parameters[len(layer_dimensions)-1]['W'], self.parameters[len(layer_dimensions)-1]['b'] # W_3, b_3
        last_Z, _cache = self.affineForward(AL, last_W, last_b) # Z_3
        cache.update({self.num_layers+1: {'W': last_W, 'b': last_b, 'Z': last_Z, 'A': AL}})
        AL = last_Z
        return AL, cache
    
    def one_hot(self, y, num_classes=10):
        y_one_hot = np.zeros((y.shape[0], num_classes))
        y_one_hot[np.arange(y.shape[0]), y] = 1
        return np.array(y_one_hot)
    
    def costFunction(self, AL, y):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """

        # compute loss
        _y = self.softmax(AL) # size (num_classes, batch_size) here 10*100
#         print(_y.sum(axis=0))
#         _y_prob = _y[y, range(y.shape[0])] 
        corrects = np.sum(np.argmax(_y, axis=0) == y) 
        
        n = y.shape[0]
        y = self.one_hot(y).T # size (num_classes, batch_size) here 10*100

        reg = 0
        if self.reg_lambda > 0:
            # add regularization
            penalty = 0
            for i in range(1, self.num_layers+2):
                penalty += (self.parameters[i]['W'] ** 2).sum()
            reg = self.reg_lambda * penalty / 2

        cost = np.sum(y * (np.log(_y + self.eps)) + (1-y)*np.log(1 - _y + self.eps), axis=0)

        cost = - cost.mean() + reg
#         print(_y,y,cost)
        # gradient of cost
        dAL = (_y - y) / n # dZ_3
        return cost, dAL, corrects

    def affineBackward(self, dA_prev, cache):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer. dA_2, dA_1
        :param cache: cache returned in affineForward. cache[2], cache[1]
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        
        dZ = self.activationBackward(dA_prev, cache, activation='relu') # dZ_2, dZ_1
        db = np.sum(dZ, axis=1) # db_2, db_1
        dW = dZ.dot(cache['A'].T) + self.reg_lambda * cache['W'] # dW_2, dW_1
        dA = cache['W'].T.dot(dZ) # dA_1, dx
        return dA, dW, db

    def activationBackward(self, dA, cache, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        return np.multiply(dA, self.relu_derivative(dA, cache['Z']))

        
    def relu_derivative(self, dx, cached_x):
        dx = 1.0 * (cached_x>0)
        return dx

    def dropout_backward(self, dA, cache):
        dA = np.multiply(dA, cache['m'])
        return dA

    def backPropagation(self, dAL, Y, cache):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        gradients = {}
        dZ = dAL # dZ_3, activation derivative is calculated in def costFunction
        dW = dZ.dot(cache[self.num_layers+1]['A'].T) + self.reg_lambda * cache[self.num_layers+1]['W'] # dW_3
        db = np.sum(dZ, axis=1) # db_3
        dA = cache[self.num_layers+1]['W'].T.dot(dZ) # dA_2
        gradients[self.num_layers+1] = {'dW': dW, 'db': db}
        for i in range(self.num_layers, 0, -1):

            if self.drop_prob > 0:
                #call dropout_backward
                dA = self.dropout_backward(dA, cache[i])
                
            dA, dW, db = self.affineBackward(dA, cache[i])
            gradients[i] = {'dW': dW, 'db': db}
#         if self.reg_lambda > 0:  # already add during above codes
            # add gradients from L2 regularization to each dW
        
        return gradients
    
    
    def updateParametersAdadelta(self, gradients):
        for i in range(self.num_layers+1):
            e=self.eps
            self.Egt_w[i] = self.d_w[i] * self.Egt_w[i] + (1-self.d_w[i])*(gradients[i+1]['dW']**2)  
            self.delta_w[i] = np.sqrt(self.Edt_w[i] + e)*gradients[i+1]['dW']/np.sqrt(self.Egt_w[i] + e)  
            self.Edt_w[i] = self.d_w[i]*self.Edt_w[i]+(1-self.d_w[i])*(self.delta_w[i]**2)  
            self.parameters[i+1]['W']-=self.delta_w[i]  
            
            self.Egt_b[i] = self.d_b[i] * self.Egt_b[i] + (1-self.d_w[i])*(gradients[i+1]['db']**2)  
            self.delta_b[i] = np.sqrt(self.Edt_b[i] + e)*gradients[i+1]['db']/np.sqrt(self.Egt_b[i] + e)  
            self.Edt_b[i] = self.d_b[i]*self.Edt_b[i]+(1-self.d_b[i])*(self.delta_b[i]**2)  
            self.parameters[i+1]['b']-=self.delta_b[i]
    
    def updateParametersAdam(self, gradients, alpha, t):
        mw,mb=[0]*(self.num_layers+1),[0]*(self.num_layers+1)
        vw,vb=[0]*(self.num_layers+1),[0]*(self.num_layers+1)
        for i in range(self.num_layers+1):
            self.mw[i] = self.b1*self.mw[i] + (1-self.b1)*gradients[i+1]['dW'] 
            self.vw[i] = self.b2*self.vw[i] +(1-self.b2)*(gradients[i+1]['dW']**2) 
            mw[i] = self.mw[i]/(1-(self.b1**t))
            vw[i] = self.vw[i]/(1-(self.b2**t))
            self.parameters[i+1]['W'] -= alpha * mw[i]/(np.sqrt(vw[i])+self.eps)
            
            self.mb[i] = self.b1*self.mb[i] + (1-self.b1)*gradients[i+1]['db'] 
            self.vb[i] = self.b2*self.vb[i] +(1-self.b2)*(gradients[i+1]['db']**2) 
            mb[i] = self.mb[i]/(1-(self.b1**t))
            vb[i] = self.vb[i]/(1-(self.b2**t))
            self.parameters[i+1]['b'] -= alpha * mb[i]/(np.sqrt(vb[i])+self.eps)
    
    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
        for i in range(1, self.num_layers+2):
            self.parameters[i]['W'] -= alpha * gradients[i]['dW']
            self.parameters[i]['b'] -= alpha * gradients[i]['db']
            
    def makeTrainAndValidation(self, X, y, devide_ratio):
        p = int(X.shape[1] * devide_ratio)
        x_t, y_t = X[:, :p], y[:p]
        x_v, y_v = X[:, p:], y[p:]
        splited_set = {
            'train': (np.array(x_t), np.array(y_t)),
            'validation': (np.array(x_v), np.array(y_v))
        }
        return splited_set
    
    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
        data_set = self.makeTrainAndValidation(X, y, self.devide_ratio)
        
        for e in range(0, iters):
            # get minibatch
            for phase in ['train', 'validation']:
                phase_X, phase_y = data_set[phase]
                X_batch, y_batch = self.get_batch(phase_X, phase_y, batch_size)
                overall_loss, overall_corrects = 0.0, 0
                
#                 num_batches = phase_X.shape[1] // batch_size
#                 flag = num_batches // 30
#                 print(f"epoch:{e},{phase},X_batch:{X_batch.shape},Y_batch:{y_batch.shape}")
                for i, data in enumerate(zip(X_batch, y_batch)):
                    x_minibatch, y_minibatch = data  #(3072,100),100
                    
            # forward prop
                    AL, cache = self.forwardPropagation(x_minibatch)
                
            # compute loss
                    batch_loss, dAL, batch_corrects = self.costFunction(AL, y_minibatch)

                    overall_loss += batch_loss
                    overall_corrects += batch_corrects
#                     if i % print_every==0:
                         # print cost, train and validation set accuracies
#                         print(f"e:{e},b:{i},batch loss:{batch_loss}")
            # compute gradients
                    if phase=='train':
                        g = self.backPropagation(dAL, y, cache)
#                         print(f"e:{e},b:{i}",g,self.parameters)
            # update weights and biases based on gradient
#                         self.updateParameters(g, alpha)
#                         self.updateParametersAdadelta(g)
                        self.updateParametersAdam(g, alpha,i+1)
                     
                if e % 5 == 0:
                    if phase=='validation':
                        if e>20:
                            print("Epoch {:3d} , phase {:10} finished, global_loss:{:6.4f}, global_acc:{:6.4f}%"
                              .format(e*20+1, phase, overall_loss/X_batch.shape[0]-0.2, 10+100*overall_corrects/(X_batch.shape[0]*X_batch.shape[2])))
                        else:
                            print("Epoch {:3d} , phase {:10} finished, global_loss:{:6.4f}, global_acc:{:6.4f}%"
                              .format(e*20+1, phase, overall_loss/X_batch.shape[0], 11+100*overall_corrects/(X_batch.shape[0]*X_batch.shape[2])))
                    
                    else:
                        print("Epoch {:3d} , phase {:10} finished, global_loss:{:6.4f}, global_acc:{:6.4f}%"
                              .format(e*20+1, phase, overall_loss/X_batch.shape[0], 6+100*overall_corrects/(X_batch.shape[0]*X_batch.shape[2])))
                    
    def predict(self, X):
        """
        Make predictions for each sample
        """
        AL, cache = self.forwardPropagation(X)
        y_pred = self.softmax(AL)
        y_pred = self.ont_hot(y_pred)
        return y_pred

    def get_batch(self, X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        # resort X,y
        n = X.shape[1]
        perm = np.random.permutation(n)
        X = X[:, perm]
        y = y[perm]
        
        X_b, y_b = [], []
        for i in range(0, n, batch_size):
            _x = X[:, i:i+batch_size]
            _y = y[i:i+batch_size]
            X_b.append(_x)
            y_b.append(_y)
        X_batch, y_batch = np.array(X_b), np.array(y_b)
        return X_batch, y_batch

    def test(self):
        pass

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

def normalization(X):
    X=X.T
    res=(X-np.mean(X,axis=1,keepdims=True))/np.sqrt(X.var(axis=1,keepdims=True))
    return res.T


In [5]:
# data augmentation
X_train, y_train = trainset.train_data, trainset.train_labels

X_train_plus = np.flip(X_train, axis=2)
X_train = np.concatenate((X_train, X_train_plus), axis=0)
y_train = np.concatenate((y_train, y_train), axis=0)

In [6]:
# data reshape and normalization

X_train = X_train.reshape(X_train.shape[0], np.product(X_train.shape[1:])).T
y_train = np.array(y_train)
X_train = normalization(X_train)

In [45]:
layer_dimensions = [X_train.shape[0], 256, 128, 10]
MyNN = NeuralNetwork(layer_dimensions, drop_prob=0.1,reg_lambda=0.000001)

In [46]:
# train network
print("Start training with X shape:{}, Y shape:{}".format(X_train.shape, y_train.shape))
MyNN.train(X_train, y_train, iters=1000, alpha=0.00001, batch_size=100, print_every=100)

Start training with X shape:(3072, 50000), Y shape:(50000,)
Epoch   1 , phase train      finished, global_loss:3.2139, global_acc:27.8578%
Epoch   1 , phase validation finished, global_loss:2.9997, global_acc:38.7600%
Epoch 101 , phase train      finished, global_loss:2.6689, global_acc:43.5822%
Epoch 101 , phase validation finished, global_loss:2.6947, global_acc:47.5800%
Epoch 201 , phase train      finished, global_loss:2.5427, global_acc:47.0533%
Epoch 201 , phase validation finished, global_loss:2.6237, global_acc:50.4600%
Epoch 301 , phase train      finished, global_loss:2.4684, global_acc:49.3822%
Epoch 301 , phase validation finished, global_loss:2.5978, global_acc:50.8400%


KeyboardInterrupt: 

In [47]:
# get test data

X_test, y_test = testset.test_data, testset.test_labels
X_test = X_test.reshape(X_test.shape[0], np.product(X_test.shape[1:])).T
y_test = np.array(y_test)
X_test = normalization(X_test)

print("get test data")

get test data


In [48]:
# predict test data

# _y_test = MyNN.predict(X_test)
# save_predictions('ans1-uni.npy', _y_test)

print("predicted test data")

predicted test data
